In [ ]:
#%matplotlib qt

In [1]:
%matplotlib notebook
%matplotlib inline
%autosave 120

Autosaving every 120 seconds


In [2]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from microsim.bp_treatment_strategies import *

from microsim.outcome_model_repository import OutcomeModelRepository
from microsim.outcome_model_type import OutcomeModelType
from microsim.person import Person

from microsim.population import NHANESDirectSamplePopulation

from microsim.bp_treatment_strategies import SprintTreatment

from microsim.sim_settings import simSettings

from microsim.trials.trialset import TrialsetParallel, TrialsetSerial
from microsim.trials.trial_description import TrialDescription
from microsim.trials.logistic_regression_analysis import LogisticRegressionAnalysis
from microsim.trials.linear_regression_analysis import LinearRegressionAnalysis
from microsim.trials.outcome_assessor import OutcomeAssessor
from microsim.trials.attribute_outcome_assessor import AttributeOutcomeAssessor
from microsim.trials.attribute_outcome_assessor import AssessmentMethod
from microsim.trials.risk_filter import RiskFilter
from microsim.trials.trial_utils import get_analysis_name, randomizationSchema

In [4]:
#any microsim dir will work, just need to access the NHANES data
#microsimDir = "/users/PAS2164/deligkaris/MICROSIM/CODE/microsim"
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)
imputedDataset = pd.read_stata(microsimDir + "/microsim/data/fullyImputedDataset.dta")
dataDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/NOTEBOOKS/DATA"

In [5]:
quantilesToInclude = pd.read_csv(dataDir + "/dementiaAndCVQuantilesToInclude.csv",index_col=0)
quantiles = pd.read_csv(dataDir + "/dementiaAndCVQuantileThresholds.csv",index_col=0)

In [6]:
#quantilesToInclude

In [7]:
inputDemThresholds=[]
inputCvThresholds=[]
for i,j in quantilesToInclude[["demRisk","cvRisk"]].iterrows():
    inputDemThresholds.append(quantiles.iloc[j["demRisk"]]["dementia"])
    inputCvThresholds.append(quantiles.iloc[j["cvRisk"]]["cv"])

In [8]:
#at the beginning turn on pandarallel if you are planning to use TrialsetSerial
pandarallel.initialize(verbose=1) #microsim by default now does not initialize pandarallel
simSettings.pandarallelFlag = True #with this flag all new population instances will be set to use pandarallel

In [10]:
%%time
# 1.5 hours 150,000 pop and 20 years
# 4 hours 150k pop and 200 years
popSprint = NHANESDirectSamplePopulation(150000, 1999)
popSprint.set_bp_treatment_strategy(SprintTreatment())
aliveSprint, dfSprint = popSprint.advance_vectorized(200)

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/pandarallel/data_types/series.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield data[chunk_]
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/Clou

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constan

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constan

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


CPU times: user 3h 1min 51s, sys: 13min 46s, total: 3h 15min 37s
Wall time: 3h 56min 3s


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


In [12]:
%%time
# 1 hour 10 min 150,000 and 20 years
# 3 hours 15 min 150k pop and 200 years
popUsual = NHANESDirectSamplePopulation(150000, 1999)
#popUsual.set_bp_treatment_strategy(SprintTreatment())
aliveUsual, dfUsual = popUsual.advance_vectorized(200)

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/pandarallel/data_types/series.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield data[chunk_]
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constant * (x.creatinine / crThreshold)

/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/gfr_equation.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return constant * (x.creatinine / crThreshold) ** exponent * 0.993**x.age


CPU times: user 1h 55min 53s, sys: 17min, total: 2h 12min 53s
Wall time: 3h 15min 27s


This is an approach based on person-years

In [13]:
def getDFForPersonCol(person):
    df = pd.DataFrame(person.get_final_wave_state_as_dict())
    df['dfIndex'] = person.dfIndex
    df['cvRisk'] = df.apply(OutcomeModelRepository().get_risk_for_person, 
                            args=(OutcomeModelType.CARDIOVASCULAR, 1, True), axis='columns')
    df['demRisk'] = df.apply(OutcomeModelRepository().get_risk_for_person,  
                             args=(OutcomeModelType.DEMENTIA, 1, True), axis='columns')
    return df

dfsUsual = popUsual._people.apply(getDFForPersonCol)
dfsSprint = popSprint._people.apply(getDFForPersonCol)

In [ ]:
allAgesUsualDF = pd.concat(dfsUsual.tolist())
allAgesSprintDF = pd.concat(dfsSprint.tolist())

In [ ]:
allAgesUsualDF["demRisk"].describe()

In [ ]:
allAgesUsualDF.loc[allAgesUsualDF["demRisk"]>1]

In [ ]:
allAgesUsualDF.head(200)

In [ ]:
allAgesUsualDF["mi"]==True

In [ ]:
allAgesUsualDF['demRiskQuantile'] = pd.cut(allAgesUsualDF["demRisk"],quantiles["dementia"])
allAgesUsualDF['cvRiskQuantile'] = pd.cut(allAgesUsualDF["cvRisk"],quantiles["cv"])

allAgesSprintDF['demRiskQuantile'] = pd.cut(allAgesSprintDF["demRisk"],quantiles["dementia"])
allAgesSprintDF['cvRiskQuantile'] = pd.cut(allAgesSprintDF["cvRisk"],quantiles["cv"])

In [ ]:
allAgesUsualDF['demRiskQuantile'].isna().sum(), allAgesUsualDF['cvRiskQuantile'].isna().sum()

In [ ]:
allAgesSprintDF['demRiskQuantile'].isna().sum(), allAgesSprintDF['cvRiskQuantile'].isna().sum()

In [ ]:
demQuantileNumberForQuantile

In [ ]:
#these are the same for both usual and sprint, the quantile intervals were simply read in this notebook
#they were created in cvDementiaQuantiles
demQuantileNumberForQuantile = {quantile : index for index, quantile in 
                                enumerate(allAgesUsualDF['demRiskQuantile'].value_counts(sort=False).index)}
cvQuantileNumberForQuantile = {quantile : index for index, quantile in 
                               enumerate(allAgesUsualDF['cvRiskQuantile'].value_counts(sort=False).index)}

allAgesUsualDF['cvQuantileNum'] = allAgesUsualDF['cvRiskQuantile']
allAgesUsualDF['cvQuantileNum'].replace(cvQuantileNumberForQuantile, inplace=True)

allAgesUsualDF['demQuantileNum'] = allAgesUsualDF['demRiskQuantile']
allAgesUsualDF['demQuantileNum'].replace(demQuantileNumberForQuantile, inplace=True)

allAgesSprintDF['cvQuantileNum'] = allAgesSprintDF['cvRiskQuantile']
allAgesSprintDF['cvQuantileNum'].replace(cvQuantileNumberForQuantile, inplace=True)

allAgesSprintDF['demQuantileNum'] = allAgesSprintDF['demRiskQuantile']
allAgesSprintDF['demQuantileNum'].replace(demQuantileNumberForQuantile, inplace=True)

In [ ]:
countByDementiaUsual = allAgesUsualDF.loc[allAgesUsualDF["dementia"]==True].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["dementia"].count().unstack(fill_value=0).stack()

countByDementiaDFUsual = pd.DataFrame({'demRisk' : countByDementiaUsual.index.get_level_values(0),
                              'cvRisk' : countByDementiaUsual.index.get_level_values(1),
                              'demCount' : countByDementiaUsual.values})

countByDementiaSprint = allAgesSprintDF.loc[allAgesSprintDF["dementia"]==True].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["dementia"].count().unstack(fill_value=0).stack()

countByDementiaDFSprint = pd.DataFrame({'demRisk' : countByDementiaSprint.index.get_level_values(0),
                              'cvRisk' : countByDementiaSprint.index.get_level_values(1),
                              'demCount' : countByDementiaSprint.values})

In [ ]:
allAgesUsualDF.loc[allAgesUsualDF["dementia"]==True].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["dementia"].count().unstack(fill_value=0).stack()

In [ ]:
countByMIUsual = allAgesUsualDF.loc[allAgesUsualDF["mi"]==True].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["mi"].count().unstack(fill_value=0).stack()

countByMIDFUsual = pd.DataFrame({'demRisk' : countByMIUsual.index.get_level_values(0),
                              'cvRisk' : countByMIUsual.index.get_level_values(1),
                              'demCount' : countByMIUsual.values})

countByMISprint = allAgesSprintDF.loc[allAgesSprintDF["mi"]==True].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["mi"].count().unstack(fill_value=0).stack()

countByMIDFSprint = pd.DataFrame({'demRisk' : countByMISprint.index.get_level_values(0),
                              'cvRisk' : countByMISprint.index.get_level_values(1),
                              'demCount' : countByMISprint.values})

In [ ]:
nQuantiles=quantileDF.shape[0]-1
#used in plots later
X=np.arange(0,nQuantiles,1)
Y=np.arange(0,nQuantiles,1)
X, Y = np.meshgrid(X, Y)
X.shape

In [ ]:
#this is done just in case the quantile categories are not ordered properly
countByDementiaArrayUsual=np.zeros(X.shape)
for i,j,k in zip(countByDementiaDFUsual["demRisk"].values,
                 countByDementiaDFUsual["cvRisk"].values,
                 countByDementiaDFUsual["demCount"].values):
    countByDementiaArrayUsual[i,j]=k

countByDementiaArraySprint=np.zeros(X.shape)
for i,j,k in zip(countByDementiaDFSprint["demRisk"].values,
                 countByDementiaDFSprint["cvRisk"].values,
                 countByDementiaDFSprint["demCount"].values):
    countByDementiaArraySprint[i,j]=k

In [ ]:
countByMIArrayUsual=np.zeros(X.shape)
for i,j,k in zip(countByMIDFUsual["demRisk"].values,
                 countByMIDFUsual["cvRisk"].values,
                 countByMIDFUsual["demCount"].values):
    countByMIArrayUsual[i,j]=k

countByMIArraySprint=np.zeros(X.shape)
for i,j,k in zip(countByMIDFSprint["demRisk"].values,
                 countByMIDFSprint["cvRisk"].values,
                 countByMIDFSprint["demCount"].values):
    countByMIArraySprint[i,j]=k

In [ ]:
countByDementiaArrayDiff = countByDementiaArrayUsual-countByDementiaArraySprint

In [ ]:
countByMIArrayDiff = countByMIArrayUsual-countByMIArraySprint

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolormesh(X, Y, countByDementiaArrayDiff, shading='nearest')
ax.set_title('difference of person-years with dementia')
plt.xlabel("cv risk quantile")
plt.ylabel("dem risk quantile")
# set the limits of the plot to the limits of the data
ax.axis([0,nQuantiles-1, 0,nQuantiles-1])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolormesh(X, Y, countByMIArrayDiff, shading='nearest')
ax.set_title('difference of person-years with mi')
plt.xlabel("cv risk quantile")
plt.ylabel("dem risk quantile")
# set the limits of the plot to the limits of the data
ax.axis([0,nQuantiles-1, 0,nQuantiles-1])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
demCountSum = countByDementiaDFUsual["demCount"].sum() #sum of all person-ages
demCountSumPerQbox = demCountSum/(nQuantiles)/(nQuantiles) # of person-ages in one qq category, uniform ditribution
#demCountSum, demCountSumPerQbox

percentCutoffAbsEffectSize = 0.05 #relevant to a uniform distribution, which qq categories to keep
cutoffDementia = (demCountSumPerQbox * percentCutoffAbsEffectSize)
#cutoff

#qqcategoriesToIncludeDementia = countByDementiaDFUsual.loc[countByDementiaDFUsual["demCount"] > cutoffDementia] 
#qqcategoriesToInclude, 
#qqcategoriesToIncludeDementia["demCount"].sum()/demCountSum #percent of person-ages included

# decide on which quantiles to keep
#includedDementia = np.zeros(X.shape)

#for i, j in zip(qqcategoriesToIncludeDementia["demRisk"],qqcategoriesToIncludeDementia["cvRisk"]):
#    includedDementia[i,j]=1
    
includedDementia = (countByDementiaArrayDiff>cutoffDementia).astype(int)

In [ ]:
#(countByDementiaArrayDiff>cutoffDementia).astype(int)

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolormesh(X, Y, includedDementia, shading='nearest')
ax.set_title('1.0 for categories to keep, 0 otherwise')
plt.xlabel("cv risk quantile")
plt.ylabel("dem risk quantile")
# set the limits of the plot to the limits of the data
ax.axis([0,nQuantiles-1, 0,nQuantiles-1])
fig.colorbar(c, ax=ax)

plt.show()

a different approach now, this one counts events based on people (not person-years)

In [ ]:
#import pickle
#with open(dataDir + "/demQuantileNumberForQuantile.pickle","rb") as file:
#    demQuantileNumberForQuantile=pickle.load(file)
#with open(dataDir + "/cvQuantileNumberForQuantile.pickle","rb") as file:
#    cvQuantileNumberForQuantile=pickle.load(file)

In [ ]:
nQuantiles=quantiles.shape[0]-1
#used in plots later
X=np.arange(0,nQuantiles,1)
Y=np.arange(0,nQuantiles,1)
X, Y = np.meshgrid(X, Y)
X.shape

In [ ]:
cvResultsUsual = popUsual._people.apply(OutcomeAssessor([OutcomeType.MI,OutcomeType.STROKE]).get_outcome)
cvResultsSprint = popSprint._people.apply(OutcomeAssessor([OutcomeType.MI,OutcomeType.STROKE]).get_outcome)

demResultsUsual = popUsual._people.apply(OutcomeAssessor([OutcomeType.DEMENTIA]).get_outcome)
demResultsSprint = popSprint._people.apply(OutcomeAssessor([OutcomeType.DEMENTIA]).get_outcome)

In [ ]:
cvRisksUsual = popUsual._people.apply(
                    OutcomeModelRepository().get_risk_for_person, 
                    args=(OutcomeModelType.CARDIOVASCULAR, 1, False))

demRisksUsual = popUsual._people.apply(
                    OutcomeModelRepository().get_risk_for_person, 
                    args=(OutcomeModelType.DEMENTIA, 1, False))

cvRisksSprint = popSprint._people.apply(
                    OutcomeModelRepository().get_risk_for_person, 
                    args=(OutcomeModelType.CARDIOVASCULAR, 1, False))

demRisksSprint = popSprint._people.apply(
                    OutcomeModelRepository().get_risk_for_person, 
                    args=(OutcomeModelType.DEMENTIA, 1, False))

In [ ]:
resultsUsual = pd.concat([cvRisksUsual,
                          demRisksUsual,
                          cvResultsUsual.astype(int),
                          demResultsUsual.astype(int)],
                         axis=1)

resultsUsual = resultsUsual.rename(columns={0:"cvRisk", 
                                            1:"demRisk",
                                            2:"cv", 
                                            3:"dementia"})
resultsSprint = pd.concat([cvRisksSprint,
                           demRisksSprint,
                           cvResultsSprint.astype(int),
                           demResultsSprint.astype(int)],
                          axis=1)
resultsSprint = resultsSprint.rename(columns={0:"cvRisk", 
                                              1:"demRisk",
                                              2:"cv", 
                                              3:"dementia"})

In [ ]:
resultsUsual.loc[resultsUsual["demRisk"]>1,"demRisk"]=1

In [ ]:
resultsUsual['demRiskQuantile'] = pd.cut(resultsUsual["demRisk"],quantiles["dementia"])
resultsUsual['cvRiskQuantile'] = pd.cut(resultsUsual["cvRisk"],quantiles["cv"])

resultsSprint['demRiskQuantile'] = pd.cut(resultsSprint["demRisk"],quantiles["dementia"])
resultsSprint['cvRiskQuantile'] = pd.cut(resultsSprint["cvRisk"],quantiles["cv"])

In [ ]:
demQuantileNumberForQuantileUsual = {quantile : index for index, quantile in 
                                enumerate(resultsUsual['demRiskQuantile'].value_counts(sort=False).index)}
cvQuantileNumberForQuantileUsual = {quantile : index for index, quantile in 
                               enumerate(resultsUsual['cvRiskQuantile'].value_counts(sort=False).index)}

demQuantileNumberForQuantileSprint = {quantile : index for index, quantile in 
                                enumerate(resultsSprint['demRiskQuantile'].value_counts(sort=False).index)}
cvQuantileNumberForQuantileSprint = {quantile : index for index, quantile in 
                               enumerate(resultsSprint['cvRiskQuantile'].value_counts(sort=False).index)}

In [ ]:
resultsUsual['cvQuantileNum'] = resultsUsual['cvRiskQuantile']
resultsUsual['cvQuantileNum'].replace(cvQuantileNumberForQuantileUsual, inplace=True)

resultsUsual['demQuantileNum'] = resultsUsual['demRiskQuantile']
resultsUsual['demQuantileNum'].replace(demQuantileNumberForQuantileUsual, inplace=True)

resultsSprint['cvQuantileNum'] = resultsSprint['cvRiskQuantile']
resultsSprint['cvQuantileNum'].replace(cvQuantileNumberForQuantileSprint, inplace=True)

resultsSprint['demQuantileNum'] = resultsSprint['demRiskQuantile']
resultsSprint['demQuantileNum'].replace(demQuantileNumberForQuantileSprint, inplace=True)

In [ ]:
countByDementiaUsual = resultsUsual.loc[resultsUsual["dementia"]==1].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["dementia"].count().unstack(fill_value=0).stack()

countByDementiaDFUsual = pd.DataFrame({'demRisk' : countByDementiaUsual.index.get_level_values(0),
                              'cvRisk' : countByDementiaUsual.index.get_level_values(1),
                              'demCount' : countByDementiaUsual.values})

countByDementiaSprint = resultsSprint.loc[resultsSprint["dementia"]==1].groupby(
                                ['cvQuantileNum', 'demQuantileNum'],sort=True
                                )["dementia"].count().unstack(fill_value=0).stack()

countByDementiaDFSprint = pd.DataFrame({'demRisk' : countByDementiaSprint.index.get_level_values(0),
                              'cvRisk' : countByDementiaSprint.index.get_level_values(1),
                              'demCount' : countByDementiaSprint.values})

In [ ]:
#this is done just in case the quantile categories are not ordered properly
countByDementiaArrayUsual=np.zeros(X.shape)
for i,j,k in zip(countByDementiaDFUsual["demRisk"].values,
                 countByDementiaDFUsual["cvRisk"].values,
                 countByDementiaDFUsual["demCount"].values):
    countByDementiaArrayUsual[i,j]=k

countByDementiaArraySprint=np.zeros(X.shape)
for i,j,k in zip(countByDementiaDFSprint["demRisk"].values,
                 countByDementiaDFSprint["cvRisk"].values,
                 countByDementiaDFSprint["demCount"].values):
    countByDementiaArraySprint[i,j]=k

In [ ]:
countByDementiaArrayDiff = countByDementiaArrayUsual-countByDementiaArraySprint

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolormesh(X, Y, countByDementiaArrayDiff, shading='nearest')
ax.set_title('difference of persons with dementia')
plt.xlabel("cv risk quantile")
plt.ylabel("dem risk quantile")
# set the limits of the plot to the limits of the data
ax.axis([0,nQuantiles-1, 0,nQuantiles-1])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
demCountSum = countByDementiaDFUsual["demCount"].sum() #sum of all person-ages
demCountSumPerQbox = demCountSum/(nQuantiles)/(nQuantiles) # of person-ages in one qq category, uniform ditribution
#demCountSum, demCountSumPerQbox

percentCutoffAbsEffectSize = 0.1 #relevant to a uniform distribution, which qq categories to keep
cutoffDementia = (demCountSumPerQbox * percentCutoffAbsEffectSize)
#cutoff

#qqcategoriesToIncludeDementia = countByDementiaDFUsual.loc[countByDementiaDFUsual["demCount"] > cutoffDementia] 
#qqcategoriesToInclude, 
#qqcategoriesToIncludeDementia["demCount"].sum()/demCountSum #percent of person-ages included

# decide on which quantiles to keep
#includedDementia = np.zeros(X.shape)

#for i, j in zip(qqcategoriesToIncludeDementia["demRisk"],qqcategoriesToIncludeDementia["cvRisk"]):
#    includedDementia[i,j]=1
    
includedDementia = (countByDementiaArrayDiff>cutoffDementia).astype(int)

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolormesh(X, Y, includedDementia, shading='nearest')
ax.set_title('1.0 for categories to keep, 0 otherwise')
plt.xlabel("cv risk quantile")
plt.ylabel("dem risk quantile")
# set the limits of the plot to the limits of the data
ax.axis([0,nQuantiles-1, 0,nQuantiles-1])
fig.colorbar(c, ax=ax)

plt.show()

In [ ]:
anyEvent = LogisticRegressionAnalysis(
                        OutcomeAssessor([
                            OutcomeAssessor.DEATH, 
                            OutcomeType.STROKE, 
                            OutcomeType.MI, 
                            OutcomeType.DEMENTIA, 
                            OutcomeAssessor.CI]))
cogEvent = LogisticRegressionAnalysis(
                        OutcomeAssessor([
                            OutcomeAssessor.CI,
                            OutcomeType.DEMENTIA]))
vascularEventOrDeath = LogisticRegressionAnalysis(
                            OutcomeAssessor([
                                OutcomeAssessor.DEATH, 
                                OutcomeType.STROKE, 
                                OutcomeType.MI]))
#anyEvent = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI, OutcomeType.DEMENTIA]))
death = LogisticRegressionAnalysis(
                            OutcomeAssessor([
                                OutcomeAssessor.DEATH]))
qalys = LinearRegressionAnalysis(
                            AttributeOutcomeAssessor(
                                "_qalys", AssessmentMethod.SUM))
meanGCP = LinearRegressionAnalysis(
                            AttributeOutcomeAssessor(
                                "_gcp", AssessmentMethod.MEAN))
lastGCP = LinearRegressionAnalysis(
                            AttributeOutcomeAssessor(
                                "_gcp", AssessmentMethod.LAST))

In [ ]:
#results.to_csv("~/Desktop/resultstest.csv")
#results